SQL Queries by Mason Howes -- Flight dataset manipulation, dataset hosted [here](https://courses.cs.washington.edu/courses/cse414/19au/hw/flight-dataset.zip)

This SQL code was originally ran with [SQLite 3.](https://www.sqlite.org/)

Set up:

In [ ]:
PRAGMA foreign_keys=ON;

-- var def
CREATE TABLE FLIGHTS (fid int PRIMARY KEY,
                month_id int,        -- 1-12
                day_of_month int,    -- 1-31
                day_of_week_id int,  -- 1-7, 1 = Monday, 2 = Tuesday, etc
                carrier_id varchar(7),
                flight_num int,
                origin_city varchar(34),
                origin_state varchar(47),
                dest_city varchar(34),
                dest_state varchar(46),
                departure_delay int, -- in mins
                taxi_out int,        -- in mins
                arrival_delay int,   -- in mins
                canceled int,        -- 1 means canceled
                actual_time int,     -- in mins
                distance int,        -- in miles
                capacity int,
                price int,           -- in $
                FOREIGN KEY (carrier_id) REFERENCES CARRIERS(cid),
                FOREIGN KEY (month_id) REFERENCES MONTHS(mid),
                FOREIGN KEY (day_of_week_id) REFERENCES WEEKDAYS(did)
                );

-- db table creation
CREATE TABLE CARRIERS (cid varchar(7), name varchar(83), PRIMARY KEY (cid));
CREATE TABLE MONTHS (mid int, month varchar(9), PRIMARY KEY (mid));
CREATE TABLE WEEKDAYS (did int, day_of_week varchar(9), PRIMARY KEY (did));

-- csv import & styling
.mode csv
.import flight-dataset\carriers.csv CARRIERS
.import flight-dataset\months.csv MONTHS
.import flight-dataset\weekdays.csv WEEKDAYS
.import flight-dataset\flights-small.csv FLIGHTS

Lists the distinct flight numbers of all flights from Seattle to Boston by Alaska Airlines Inc. on Mondays:

In [ ]:
SELECT DISTINCT flight_num AS flight_num
FROM FLIGHTS AS f,
     CARRIERS AS c,
     WEEKDAYS AS w
WHERE f.carrier_id = c.cid
    AND f.day_of_week_id = w.did
    AND f.origin_city = 'Seattle WA'
    AND f.dest_city = 'Boston MA'
    AND c.name = 'Alaska Airlines Inc.'
    AND w.day_of_week = 'Monday';

Finds all itineraries from Seattle to Boston on July 15th:

In [ ]:
SELECT c.name AS name,
       f1.flight_num AS f1_flight_num,
       f1.origin_city AS f1_origin_city,
       f1.dest_city AS f1_dest_city,
       f1.actual_time AS f1_actual_time,
       f2.flight_num AS f2_flight_num,
       f2.origin_city AS f2_origin_city,
       f2.dest_city AS f2_dest_city,
       f2.actual_time AS f2_actual_time,
       f1.actual_time + f2.actual_time AS actual_time
FROM FLIGHTS AS f1,
     FLIGHTS AS f2,
     CARRIERS AS c,
     MONTHS AS m
WHERE f1.origin_city = 'Seattle WA'
  AND f2.dest_city = 'Boston MA'
  AND f1.dest_city = f2.origin_city
  AND f1.month_id = m.mid
  AND f2.month_id = m.mid
  AND m.month = 'July'
  AND f1.day_of_month = 15
  AND f2.day_of_month = 15
  AND f1.carrier_id = c.cid
  AND f2.carrier_id = c.cid
  AND f1.carrier_id = f2.carrier_id
  AND (f1.actual_time + f2.actual_time) < 420; -- 7hrs conv to min

Finds the day of the week with the longest average arrival delay. Returns the name of the day and the average delay:

In [ ]:
SELECT w.day_of_week AS day_of_week,
       AVG(f.arrival_delay) AS delay
FROM FLIGHTS AS f,
     WEEKDAYS AS w
WHERE f.day_of_week_id = w.did
GROUP BY w.day_of_week
ORDER BY AVG(f.arrival_delay) DESC
LIMIT 1;

Finds the names of all airlines that ever flew more than 1000 flights in one day (i.e., a specific day/month, but not any 24-hour period):

In [ ]:
SELECT DISTINCT c.name AS name
FROM FLIGHTS AS f,
     MONTHS as m,
     CARRIERS as c
WHERE f.day_of_month = m.mid
  AND f.carrier_id = c.cid
GROUP BY m.month, f.day_of_month, c.name
HAVING COUNT(*) > 1000;

Finds all airlines that had more than 0.5% (= 0.005) of their flights out of Seattle canceled. Returns the name of the airline and the percentage of canceled flights out of Seattle. Results are ordered by the percentage of canceled flights in ascending order:

In [ ]:
SELECT c.name AS name,
       AVG(f.canceled) * 100 AS percentage
FROM FLIGHTS AS f,
     CARRIERS AS c
WHERE f.carrier_id = c.cid
  AND f.origin_city = 'Seattle WA'
GROUP BY c.name
HAVING AVG(f.canceled) > 0.005;

Finds the maximum price of tickets between Seattle and New York, NY:

In [ ]:
SELECT c.name AS carrier,
       MAX(f.price) as max_price
FROM FLIGHTS AS f,
     CARRIERS AS c
WHERE f.carrier_id = c.cid
  AND ((f.origin_city = 'Seattle WA' AND f.dest_city = 'New York NY')
      OR (f.origin_city = 'New York NY' AND f.dest_city = 'Seattle WA'))
GROUP BY c.name;

Finds the total capacity of all direct flights that fly between Seattle and San Francisco, CA on July 10th:

In [ ]:
SELECT SUM(f.capacity) as capacity -- SUM() = addition, COUNT() = total rows
FROM FLIGHTS AS f,
     MONTHS AS m
WHERE f.month_id = m.mid
  AND m.month = 'July'
  AND f.day_of_month = 10
  AND ((f.origin_city = 'Seattle WA' AND f.dest_city = 'San Francisco CA') -- Seattle -> SF
      OR (f.origin_city = 'San Francisco CA' AND f.dest_city = 'Seattle WA'));  -- SF -> Seattle

Computes the total departure delay of each airline across all flights:

In [ ]:
SELECT c.name AS name,
       SUM(f.departure_delay) AS delay
FROM FLIGHTS AS f,
     CARRIERS AS c
WHERE f.carrier_id = c.cid
GROUP BY c.name;